In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
url = 'https://www.technodom.kz/'
headers = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36'}

In [3]:
productlinks = []
for x in range(1, 15):
    r = requests.get(f'https://www.technodom.kz/catalog/smartfony-i-gadzhety/smartfony-i-telefony?page={x}')
    soup = BeautifulSoup(r.content, 'lxml')
    productlist = soup.find_all('ul', class_='category-page-list__list')
    for product in productlist:
        for link in product.find_all('a', href = True):
            productlinks.append(url + link['href'])
            productlinks = set(productlinks)
            productlinks = [ x for x in productlinks if "#commentsTab" not in x ]

In [4]:
print(len(productlinks))

168


In [5]:
itemlist = []
for link in productlinks:
    r = requests.get(link, headers = headers)
    soup = BeautifulSoup(r.content, 'lxml')
    name = soup.find('p', class_='Typography product-info__mobile-title Typography__Subtitle Typography__Subtitle_Small').text.strip()
    try:
        price = soup.find('p', class_='Typography Typography__Heading Typography__Heading_H1').text.strip()
    except:
        price = 'Sold Out'
    try:
        rating = soup.find('p', class_='Typography StickerNext StickerNext__Text product-rating__sticker Typography__Caption Typography__Caption_Bold').text.strip()
    except: 
        star = 'No Ratings' 
    item = {'Name' : name,'Price' : price,'Rating out of 5' : rating}
    itemlist.append(item)

In [6]:
data = pd.DataFrame(itemlist)
data

,Name,Price,Rating out of 5
0,Смартфон Xiaomi Redmi 10C 128GB/4GB Graphite Gray,Sold Out,4.8
1,Смартфон Apple iPhone 13 Pro Max 128GB Sierra ...,Sold Out,0.0
2,Смартфон Samsung Galaxy S22 Ultra 512GB Phanto...,Sold Out,5.0
3,Смартфон Xiaomi 12 Lite 256GB/8GB Lite Pink,239 990 ₸,5.0
4,Смартфон Apple iPhone 13 Pro Max 512GB Sierra ...,Sold Out,0.0
...,...,...,...
163,Смартфон HUAWEI Nova Y70 128GB Midnight Black,Sold Out,5.0
164,Смартфон Apple iPhone 14 Plus 512GB Starlight,839 990 ₸,4.0
165,Смартфон Samsung Galaxy S22+ 256GB Phantom Black,Sold Out,5.0
166,Смартфон Samsung Galaxy S22 Ultra 128GB Green,Sold Out,4.5


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 168 entries, 0 to 167
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Name             168 non-null    object
 1   Price            168 non-null    object
 2   Rating out of 5  168 non-null    object
dtypes: object(3)
memory usage: 4.1+ KB


In [8]:
data['Name'] = data['Name'].astype('string')

In [9]:
data['Name'] = data['Name'].str.split(n=1).str[1]

In [10]:
data.head()

,Name,Price,Rating out of 5
0,Xiaomi Redmi 10C 128GB/4GB Graphite Gray,Sold Out,4.8
1,Apple iPhone 13 Pro Max 128GB Sierra Blue,Sold Out,0.0
2,Samsung Galaxy S22 Ultra 512GB Phantom White,Sold Out,5.0
3,Xiaomi 12 Lite 256GB/8GB Lite Pink,239 990 ₸,5.0
4,Apple iPhone 13 Pro Max 512GB Sierra Blue,Sold Out,0.0


In [11]:
data = data[data["Price"].str.contains("Sold Out") == False]
data.head()

,Name,Price,Rating out of 5
3,Xiaomi 12 Lite 256GB/8GB Lite Pink,239 990 ₸,5.0
6,Samsung Galaxy A03 32GB Red,69 990 ₸,5.0
9,OPPO Reno 7 256GB 5G Starry Black,259 990 ₸,5.0
11,Xiaomi 12T 256GB/8GB Silver,299 990 ₸,0.0
13,Poco M4 Pro 256/8GB 4G Power Black,149 990 ₸,5.0


In [12]:
data['Price'] = data['Price'].str.strip("₸")

C:\Users\Мерей\AppData\Local\Temp\ipykernel_33688\1507164464.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Price'] = data['Price'].str.strip("₸")


In [13]:
data['Rating out of 5'] = data['Rating out of 5'].astype('float')

C:\Users\Мерей\AppData\Local\Temp\ipykernel_33688\279904193.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Rating out of 5'] = data['Rating out of 5'].astype('float')


In [14]:
data['Price'] = data['Price'].str.strip()

C:\Users\Мерей\AppData\Local\Temp\ipykernel_33688\608253498.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Price'] = data['Price'].str.strip()


In [15]:
data.to_csv("technodom.csv", encoding="utf-8")